In [166]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

import tensorflow.keras.losses as losses

In [167]:
# Cargar datos desde CSV
dataAudio = pd.read_csv('C:/Users/LEHI/Documents/Practicas profesionales/ABRIL/DatasetAudioPrueba2.csv')

In [168]:
#Seleccion de elementos por clase
df_uniq = dataAudio.groupby('personCLASE').head(50)

In [169]:
#Vizualizar los datos seleccionados
#df_uniq

In [170]:
def eliminar_columnas(data, columnas_a_eliminar):
    dataSC= df_uniq.copy()
    dataSC=df_uniq.drop(columns=columnas_a_eliminar, errors='ignore')
    return dataSC

In [171]:
columnasE=['signal_mean_beats']

In [172]:
dataLimpiaAudio=eliminar_columnas(df_uniq,columnasE)

In [173]:
#dataLimpiaAudio

In [174]:
# Dividir los datos en características (X) y etiquetas (y)
X = dataLimpiaAudio.drop('personCLASE', axis=1)
y = dataLimpiaAudio['personCLASE']

In [175]:
# Convertir etiquetas a números
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [176]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos (opcional pero recomendado)
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

In [177]:
# Definir la arquitectura de la red siamesa
input_shape = X_train.shape[1]

In [178]:
# Definir función para crear la rama de la red siamesa
def create_siamese_branch(input_shape):
    input_layer = tf.keras.layers.Input(shape=(input_shape,))
    x = tf.keras.layers.Dense(64, activation='relu')(input_layer)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    return tf.keras.Model(inputs=input_layer, outputs=x)

In [179]:
# Crear la rama izquierda y derecha de la red siamesa
left_input = tf.keras.layers.Input(shape=(input_shape,))
right_input = tf.keras.layers.Input(shape=(input_shape,))

left_branch = create_siamese_branch(input_shape)(left_input)
right_branch = create_siamese_branch(input_shape)(right_input)

In [180]:
# Concatenar las salidas de ambas ramas
merged = tf.keras.layers.Concatenate()([left_branch, right_branch])

In [181]:
# Agregar capas adicionales y la capa de salida
x = tf.keras.layers.Dense(32, activation='relu')(merged)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(3, activation='softmax')(x)  

In [182]:
# Construir el modelo
model = tf.keras.Model(inputs=[left_input, right_input], outputs=output)

In [183]:
# Compilar el modelo
model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])


In [184]:
# Entrenar el modelo
history = model.fit([X_train, X_train], y_train, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
6/6 [==============================] - 1s 51ms/step - loss: 1.2234 - accuracy: 0.3229 - val_loss: 1.1598 - val_accuracy: 0.5000
Epoch 2/50
6/6 [==============================] - 0s 8ms/step - loss: 1.1181 - accuracy: 0.4479 - val_loss: 1.0595 - val_accuracy: 0.6250
Epoch 3/50
6/6 [==============================] - 0s 9ms/step - loss: 1.0168 - accuracy: 0.4583 - val_loss: 0.9984 - val_accuracy: 0.6250
Epoch 4/50
6/6 [==============================] - 0s 8ms/step - loss: 0.9235 - accuracy: 0.5521 - val_loss: 0.9564 - val_accuracy: 0.6667
Epoch 5/50
6/6 [==============================] - 0s 7ms/step - loss: 0.9006 - accuracy: 0.5521 - val_loss: 0.9180 - val_accuracy: 0.7083
Epoch 6/50
6/6 [==============================] - 0s 9ms/step - loss: 0.8742 - accuracy: 0.5833 - val_loss: 0.8780 - val_accuracy: 0.7917
Epoch 7/50
6/6 [==============================] - 0s 8ms/step - loss: 0.8079 - accuracy: 0.7396 - val_loss: 0.8293 - val_accuracy: 0.7917
Epoch 8/50
6/6 [=================

In [185]:
# Cargar datos desde CSV
dataAudio = pd.read_csv('C:/Users/LEHI/Documents/Practicas profesionales/ABRIL/DatasetAudioPrueba2.csv')

In [186]:
#Cuando se vuelve a cargar el dataset se deben eliminar las columnas que se eliminaron en la fase de entrenamiento
dataAudio=eliminar_columnas(dataAudio,columnasE)

In [187]:
# Dividir los datos en características (X) y etiquetas (y)
X = dataAudio.drop('personCLASE', axis=1)
y = dataAudio['personCLASE']

In [188]:
# Convertir etiquetas a números
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [189]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Normalizar los datos (opcional pero recomendado)
X_test = (X_test - X_test.mean()) / X_test.std()

In [190]:
# Evaluar el modelo
y_pred = model.predict([X_test, X_test])
y_pred_classes = np.argmax(y_pred, axis=1)

1/1 [==============================] - 0s 68ms/step


In [191]:
# Matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Matriz de Confusión:")
print(conf_matrix)

Matriz de Confusión:
[[11  1  0]
 [ 1  8  0]
 [ 0  0  9]]


In [192]:
#Metricas

with np.errstate(divide='ignore', invalid='ignore'):
    # Exactitud (Accuracy)
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    print("Exactitud (Accuracy):\n", accuracy)

    # Precisión (Precision)
    precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
    precision[np.isnan(precision)] = 0  # Reemplaza NaN con 0
    print("Precisión (Precision) por clase:\n", precision)

    # Sensibilidad (Recall)
    sensibilidad = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
    sensibilidad[np.isnan(sensibilidad)] = 0  # Reemplaza NaN con 0
    print("Sensibilidad (Recall) por clase:\n", sensibilidad)

    # Puntuación F1 (F1 Score)
    f1_score = 2 * (precision * sensibilidad) / (precision + sensibilidad)
    f1_score[np.isnan(f1_score)] = 0  # Reemplaza NaN con 0
    print("Puntuación F1 (F1 Score) por clase:\n", f1_score)

Exactitud (Accuracy):
 0.9333333333333333
Precisión (Precision) por clase:
 [0.91666667 0.88888889 1.        ]
Sensibilidad (Recall) por clase:
 [0.91666667 0.88888889 1.        ]
Puntuación F1 (F1 Score) por clase:
 [0.91666667 0.88888889 1.        ]
